code to analyze the 2-alternative forced choice questions testing for participants' knowledge of the episodic detail on day 1 throughout learning (meant to check for general engagement with the task). 

In [ ]:
import pandas as pd
import os
import numpy as np
import ast
import fnmatch
import glob
import matplotlib.pyplot as plt
import random
import seaborn

In [ ]:
def simplify_path_string(path_string):
    new_path =[]
    for i in path_string:
        if 'S' in i:
            new_path.append('2')
        if 'N' in i:
            new_path.append('1')
    return new_path   

def simplify_eggPainting_string(eggPainting_string):
    new_eggPainting=[]
    for i in eggPainting_string:
        if 'newegg' in i:
            new_eggPainting.append('2')
        if '/egg' in i:
            new_eggPainting.append('1')
        if 'newpainting' in i:
            new_eggPainting.append('2')
        if '/painting' in i:
            new_eggPainting.append('1')
    return new_eggPainting

def simplify_character_string(character_string):
    new_character=[]
    for i in character_string:
        if 'newperson' in i:
            new_character.append('2')
        if 'couple' in i:
            new_character.append('1')
    return new_character

def simplify_gift_string(gift_string):
    new_gift=[]
    for i in gift_string:
        if 'new' in i:
            new_gift.append('2')
        if 'gift' in i:
            new_gift.append('1')
    return new_gift

def simplify_name_string(name_string):
    new_name=[]
    for i in name_string:
        if 'cn' in i:
            new_name.append('2')
        if '/n' in i:
            new_name.append('1')
    return new_name

def simplify_theme_string(theme_string):
    new_theme=[]
    for i in theme_string:
        if 'ct' in i:
            new_theme.append('2')
        if '/t' in i:
            new_theme.append('1')
    return new_theme

def return_boolean(value1, value2):
    if value1 == value2:
        return 1
    else:
        return 0

In [ ]:
# load data
path = '/Volumes/norman/scollin/schema/data/behav/sub*'

files = [f for f in glob.glob(path + "**/*_Block1.csv", recursive=True)]

TotalResultsMatrixMeans = np.zeros((len(files),4))


In [ ]:
# loop over the 4 task blocks and extract the responses
for currBlock in [1,2,3,4]:
    
    path = '/Volumes/norman/scollin/schema/data/behav/sub*'

    files = [f for f in glob.glob(path + "**/*_Block%i.csv" %currBlock, recursive=True)]

    filename_list = sorted(files)

    TotalResultsMatrix = np.zeros((len(filename_list),6))

    for subjID in range(len(filename_list)):

        #TotalResultsMatrix = np.empty((56, 6))

        filename = filename_list[subjID]
        print(filename)
        answerkey = pd.read_csv(filename, usecols = ['QeggPainting1','Qcharacter1', 'Qgift1', 'Qname1', 'Qtheme1', 'condsFile'])

        newanswerkey = answerkey.iloc[[3,6,9,12,15,18]]

        path_string = newanswerkey['condsFile'].tolist()
        eggPainting_string = newanswerkey['QeggPainting1'].tolist()
        character_string = newanswerkey['Qcharacter1'].tolist()
        gift_string = newanswerkey['Qgift1'].tolist()
        name_string = newanswerkey['Qname1'].tolist()
        theme_string = newanswerkey ['Qtheme1'].tolist()

        correct_path = simplify_path_string(path_string)
        correct_eggPainting = simplify_eggPainting_string(eggPainting_string)
        correct_character = simplify_character_string(character_string)
        correct_gift = simplify_gift_string(gift_string)
        correct_name = simplify_name_string(name_string)
        correct_theme = simplify_theme_string(theme_string)

        d = {'name': correct_name, 'eggPainting': correct_eggPainting, 'character': correct_character, 'theme': correct_theme, 'gift': correct_gift, 'path':correct_path}
        df = pd.DataFrame(d)

        responses = pd.read_csv(filename, usecols = ['nameQresp.keys', 'Qcharacter.keys', 'eggPaintingQresp.keys','themeQresp.keys', 'giftQresp.keys', 'northOrSouthQ.keys'],dtype=object)
        responses = responses.dropna()

        vectorized = np.vectorize(return_boolean)
        name_column = vectorized(df['name'].tolist(), responses['nameQresp.keys'].tolist())
        character_column = vectorized(df['character'].tolist(), responses['Qcharacter.keys'].tolist())
        eggPainting_column = vectorized(df['eggPainting'].tolist(), responses['eggPaintingQresp.keys'].tolist())
        theme_column = vectorized(df['theme'].tolist(), responses['themeQresp.keys'].tolist())
        path_column = vectorized(df['path'].tolist(), responses['northOrSouthQ.keys'].tolist())
        gift_column = vectorized(df['gift'].tolist(), responses['giftQresp.keys'].tolist())

        dictionary = {'name_match': name_column, 'eggPainting_match': eggPainting_column, 'character_match':character_column, 'theme_match': theme_column, 'gift_match':gift_column, 'path_match':path_column}
        to_ret = pd.DataFrame(dictionary)

        meanData = to_ret.mean(axis = 0)

        #np.append(TotalResultsMatrix, np.array([meanData]))
        TotalResultsMatrix[subjID,:] = meanData

        fname='/Volumes/norman/scollin/schema/data/behav/results/attentionTask_Day1_block%i.csv' % (currBlock)

        #np.savetxt(fname, TotalResultsMatrix, delimiter=',')
        
    TotalResultsMatrixMeans[:,currBlock-1] = np.mean(TotalResultsMatrix, axis=1)

In [ ]:
# save the data
np.savetxt('/Volumes/norman/scollin/schema/data/behav/results/attention_Task_Day1.csv', TotalResultsMatrixMeans, delimiter=',')

